In [140]:
import pandas as pd 
import pickle
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
from sklearn.metrics import roc_curve, auc
import time
from sklearn import linear_model
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
from sklearn.metrics import r2_score
import multiprocessing

In [141]:
#Checking number of scores
multiprocessing.cpu_count()

8

In [ ]:
#Generating training data set to predict runtime of RF

In [6]:
def measure_time(n=10,p=10,i=5,j=2,k=100,l=-1):
    start_time = time.time()
    X=np.random.rand(n,p)
    y=np.random.rand(n,)
    clf = RandomForestRegressor(max_depth=i, max_features=j, n_estimators=k,n_jobs=l)
    clf.fit(X,y)
    elapsed_time = time.time() - start_time
    return elapsed_time

In [7]:
inputs=[]
outputs=[]

In [8]:
for i in [10,50,100] :
    for k in [10,50,100]:
        for p in [5,50,100]:
            for j in range(1,p,10):
                for n in [100,1000,10000]:
                    outputs.append(measure_time(j=j,i=i,n=n,k=k,p=p))
                    inputs.append(np.array([n,p,i,j,k,-1]))

In [9]:
y=np.array(outputs)
X=np.array(inputs)

In [142]:
X.shape

(432, 6)

In [143]:
y.shape

(432,)

In [114]:
#Adding n*log(n)*v (supposedly = runtime of training in big o notation)
X_1=np.append(X,np.array(X[:,1]*X[:,0]*np.log(X[:,0])).reshape(432,1),axis=1)

In [115]:
X_1.shape

(432, 7)

In [10]:
#Trying to train with a RF
cl=RandomForestRegressor(max_depth=20, max_features=5, n_estimators=100,n_jobs=-1)

In [11]:
cl.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
           max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [145]:
#Great R squared - but might be overfitting?
r2_score(y, cl.predict(X)) 

0.9981003098586998

In [149]:
#Problem : doesnt go outside of the training range... TBD
cl.predict(np.array([[c, 5,    10,     100,    10000,    -1]])) 

array([ 9.22206372])

In [116]:
#Trying linreg
regr = linear_model.LinearRegression()

In [117]:
regr.fit(X_1,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [119]:
#Decent R squared
r2_score(y, regr.predict(X_1)) 

0.54524986068417214

In [153]:
measure_time(n=10000,p=10,i=25,j=9,k=200,l=-1)

3.586660861968994

In [151]:
#seems relevant 
regr.predict(np.array([[100000000000000000, 10,25,    9,     200,    -1,100000000000000000*np.log(100000000000000000)*10]])) 

array([  1.50455252e+13])